# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
import pandas as pd

In [3]:
try:
    data = pd.read_csv('/datasets/data.csv') 
except:
    data = pd.read_csv('data.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data['days_employed'] = data['days_employed'].abs()

In [6]:
data['education'] = data['education'].str.lower()
data.isna().sum() # пропуска выявил методом isna()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**

В данной таблице присутствуют пропуски в колонках "days_employed" и "total_income" 
Пропуски обозначены как : NaN

Преобразовал все минусовые значения в столбце 'days_employed', потому что не корректно работать с минусовым стажем работы

Изменил регистр в колонке 'education'

## Шаг 2. Предобработка данных

### Обработка пропусков

In [7]:
data_pension = data[data['income_type'] == 'пенсионер']['days_employed']
data_employee = data[data['income_type'] == 'сотрудник']['days_employed']
data_companion = data[data['income_type'] == 'компаньон']['days_employed']
data_servant = data[data['income_type'] == 'госслужащий']['days_employed']

data_pension.sample(5)

21340    397408.466072
7908     338989.550143
12758    352899.571175
16448              NaN
8781     396641.810893
Name: days_employed, dtype: float64

In [8]:
data.loc[data['income_type'] == 'пенсионер','days_employed'] = data.loc[data['income_type'] == 'пенсионер','days_employed'] // 24 

При проверке многочисленных категорий типов занятости, было выявлено что у пенсионеров указан сверх большой стаж. 
У нас в таблице данные указаны в днях, если перевести стаж пенсионеров в года, то получатся нереальные значения, люди столько не живут.
Скорее всего стаж у пенсионеров указан в часах и я решил их стаж в часах перевести в дни.

In [9]:
medi_pens_day = data.loc[data['income_type'] == 'пенсионер', 'days_employed'].median()
data.loc[data['income_type'] == 'пенсионер', 'days_employed'] = data.loc[data['income_type'] == 'пенсионер', 'days_employed'].fillna(value=medi_pens_day)

medi_employ_day = data.loc[data['income_type'] == 'сотрудник', 'days_employed'].median()
data.loc[data['income_type'] == 'сотрудник', 'days_employed'] = data.loc[data['income_type'] == 'сотрудник', 'days_employed'].fillna(value=medi_employ_day)

medi_serv_day =  data.loc[data['income_type'] == 'госслужащий', 'days_employed'].median()
data.loc[data['income_type'] == 'госслужащий', 'days_employed'] = data.loc[data['income_type'] == 'госслужащий', 'days_employed'].fillna(value=medi_serv_day)

medi_comp_day = data.loc[data['income_type'] == 'компаньон', 'days_employed'].median()
data.loc[data['income_type'] == 'компаньон', 'days_employed'] = data.loc[data['income_type'] == 'компаньон', 'days_employed'].fillna(value=medi_comp_day)

In [10]:
medi_pens_income = data.loc[data['income_type'] == 'пенсионер', 'total_income'].median()
data.loc[data['income_type'] == 'пенсионер', 'total_income'] = data.loc[data['income_type'] == 'пенсионер', 'total_income'].fillna(value=medi_pens_income)

medi_employ_income = data.loc[data['income_type'] == 'сотрудник', 'total_income'].median()
data.loc[data['income_type'] == 'сотрудник', 'total_income'] = data.loc[data['income_type'] == 'сотрудник', 'total_income'].fillna(value=medi_employ_income)

medi_serv_income =  data.loc[data['income_type'] == 'госслужащий', 'total_income'].median()
data.loc[data['income_type'] == 'госслужащий', 'total_income'] = data.loc[data['income_type'] == 'госслужащий', 'total_income'].fillna(value=medi_serv_income)

medi_comp_income = data.loc[data['income_type'] == 'компаньон', 'total_income'].median()
data.loc[data['income_type'] == 'компаньон', 'total_income'] = data.loc[data['income_type'] == 'компаньон', 'total_income'].fillna(value=medi_comp_income)

Заменил пропуски на медианные значения, по категории занятости.

In [11]:
data.isna().sum()

children            0
days_employed       1
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        1
purpose             0
dtype: int64

Все остальные пропуски заменил на 0, так как эти показатели не повлияют на результат исследования.

In [ ]:
data.isna().sum()
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)

**Вывод**

В столбцах 'days_employed' и 'total_income' были пропуски NaN.
Решил их заменить на медианные значения, по категориям занятости.
Предварительно перевел некорректные данные в столбце 'days_employed' 

### Замена типа данных

In [ ]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')

In [ ]:
data.info()

**Вывод**

Также заменил типы данных в столбце 'days_employed' и 'total_income' c вещественного на колличественный (float на int)

### Обработка дубликатов

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

In [ ]:
data.loc[data['children'] == 20, 'children'] = data.loc[data['children'] == 20, 'children'] / 10

In [ ]:
data['children'] = data['children'].abs().astype('int')

In [ ]:
data['children'].value_counts()

In [ ]:
data.duplicated().sum()

**Вывод**

Дубликаты из таблицы удалены.
Также преобразовал колличество детей с минусовых значений в плюсовые
Колличество 20 детей изменил на колличество 2. 20 детей в семье встречается очень редко в мире, а вот 2 в полне часто. Скорее всего это опечатка.

### Лемматизация

In [ ]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
def lemmatize(text):
    lemmas = m.lemmatize(text)
    return (Counter(lemmas))


data['lemms'] = data['purpose'].apply(lemmatize)
lems_sum = data['lemms'].drop_duplicates().reset_index(drop=True).sum()

**Вывод**

Лемматизировал столбец 'purpose', для легкого поиска целей кредита по леммам.

### Категоризация данных

In [ ]:
lems_sum = data['lemms'].reset_index(drop=True).sum()
lems_sum

Наиболее повторяющиеся леммы это: недвижимость, образование, жилье, автомобиль, свадьба
По ним  и делаем категоризацию.

In [ ]:
def target_credit(lemma):
    
    if 'свадьба' in lemma:
        return 'свадьба'
    if 'недвижимость' in lemma:
        return 'недвижимость'
    if 'автомобиль' in lemma:
        return 'автомобиль'
    if 'жилье' in lemma:
        return 'недвижимость'
    if 'образование' in lemma:
        return 'образование'
data['target'] = data['lemms'].apply(target_credit)
data.sample(5)


In [ ]:
edu_dict = data[['education_id','education']]

fam_dict = data[['family_status_id', 'family_status']]

In [ ]:
edu_dict.drop_duplicates().reset_index()

In [ ]:
data['total_income'].quantile(.25)

In [ ]:
data['total_income'].quantile(.5)

In [ ]:
data['total_income'].quantile(.75)

In [ ]:
def income_category(q):
        if 0 <= q <= 107584:
            return 'очень низкий'
        if 107585 <= q <= 142593:
            return 'низкий'
        if 142594 <= q <= 195813:
            return 'средний'
        if 195814 <= q <= 10000000:
            return 'высокий'

data['category_income'] = data['total_income'].apply(income_category)

In [ ]:
data['category_income']

**Вывод**

Сделал категоризацию по леммам, чтоб цель кредита была яснее выражена.
Выделил четыре словаря для исследования.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data.pivot_table(index ='children', values = 'debt', aggfunc = ['count', 'sum', 'mean'])


**Вывод**

По таблице видно что больше всего семей, которые не имеют детей и колличество должников из них больше всего, но часть должников из общего колличества меньше, чем у семей у кого есть дети.
Особенно это видно там где семьи или люди имеют по одному, двум и четырем детям. 

Делаю вывод что дети это отдельная статья расходов, что создает дополнительную нагрузку на семейный бюджет. 

Поэтому в семье где есть дети, больше должников.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data.pivot_table(index ='family_status', values = 'debt', aggfunc = ['count', 'sum', 'mean'])

**Вывод**

По данной таблице видно, что большую часть должников составляют те кто "не женат / не замужем" и те кто в "гражданском браке'. 



- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data.pivot_table(index='category_income', values='debt', aggfunc=['sum', 'mean'])

**Вывод**

По данным в этой таблицы, видно что больше всего должников из тех, у кого доход низкий и средний.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data.pivot_table(index ='target', values = 'debt', aggfunc = ['count', 'sum', 'mean'])

**Вывод**

В этой таблице видно, что люди, которые  покупают недвижимость , более ответственно подходят к выплате долга ежели те, кто покупает машину в кредит или берет на образование.


## Шаг 4. Общий вывод

**Вывод**

1. Больше всего должников из семей с одним, двумя и четыремя детьми.
Делаю вывод что дети это отдельная статья расходов и это создает дополнительную нагрузку на семейный бюджет.
Поэтому в семье где есть дети, больше должников.

2. Большую часть должников составляют те кто "не женат / не замужем" и те кто в "гражданском браке'.

3. Больше всего должников из тех, у кого доход низкий и средний доход.

4. Люди, которые покупают недвижимость , более ответственно подходят к выплате долга ежели те, кто покупает машину в кредит или берет на образование.



При работе с данными были исправлены ошибки, выявлены пропуски и заменены на медианные значения, удалены дубликаты, выделены категории целей кредита с помощью лемматизации и категоризации. 

При помощи сводных таблиц я оценил какие критерии влияют на возврат кредита в срок.